In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import pickle
import os
import pandas as pd
import json
from itertools import compress
import math

from scipy import signal 

import glob

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.utils 

from DataManager import ParticipantData, MeasurementData
from Helpers import read_normalized_json_to_df, save_to_disk, load_from_disk, create_rolling_windows, \
   intersection, reject_outliers, anglebetween, eye_outlier_removal_sigma, eye_outlier_removal_zero_values, validate_participant_seen_areas
from AngleHelpers import create_relative_directions

In [ ]:
reference_data_all_failed_save_path = "./post_recorded_reference_data/reference_dict_all.pickle"
REFERENCE_DATA_ALL_EVENTS_FAILED = load_from_disk(reference_data_all_failed_save_path)

correlation_dict_path = "./post_recorded_reference_data/corr_dict.pickle"
CORRELATION_DICT = load_from_disk(correlation_dict_path)

final_data_path = "./data/"

# Load data

In [ ]:
# Exclude all participants that have less than two (complete) scenes besides the training scene
exclude_participant_ids = [
    "a5c5d2051bd1493ea79e1d565524610f", # ['MountainRoad.txt']
    "2462e5d82bc44a328934442cae03d83f", # ['MountainRoad.txt', 'TrainingScene.txt']
    "0c2734d5a8664e1abd2ffcf8aa6fa5d6", # ['TrainingScene.txt']
    "c685e206b2eb4702860bb908afe2a11f", # ['MountainRoad.txt', 'TrainingScene.txt']
    "28d1f769c1204e36a574c025d8e771c4", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "d5f3fa1c877f4afb9bc0250d5882c22c", # ['TrainingScene.txt']
    "f4c4e380b2b941e4a963fe12f4038865", # ['MountainRoad.txt', 'TrainingScene.txt']
    "1c293d21f60241008d6230a6f28e4079", # ['TrainingScene.txt']
    "3e3154cff72540e48eac982a379ce69f", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "1deaa8e567f1435da07c0817717ef4fe", # ['TrainingScene.txt']
    "28191a22a70d478b808eb8905e35c6ba", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "ac30f09446ec48289c41b66b1ba0d3f9", # ['TrainingScene.txt'] 
    "5b79b725686a4985b5caf35de0e06d43", # ['Incomplete.txt', 'TrainingScene.txt']
    "b482259cbb524e689a6e3859951bebf1", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "518e07b76d804110b035e3c774f52d5b", # ['TrainingScene.txt']
    "cc9284dfb5f645858196537e05c1959b", # ['MountainRoad.txt']
    "0cd51e7040bc43568284cdcedccfe0d3", # ['TrainingScene.txt']
    "d9cd9704a0064fed9993333363608346", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "5ff5cecd56f7486286ec386434ae9219", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "178dd53ce37a40b0accd7a59cc9035ac", # ['TrainingScene.txt']
    "1efd4163b5614c8e9eed8e04c0ceda3e", # ['MountainRoad.txt', 'TrainingScene.txt'] 
    "b74d6990eba645fa82fa2ab9fb0b5180", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "711fa186ae5e4d9fada5b0e0049a2074", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "83b00eb628b2446d992ac698026dfdc0", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "f2fb2a62a3ca4033b9005be13842e27c", # ['MountainRoad.txt']
    "0c0c45348e314379994333127e5456ae", # ['TrainingScene.txt']
    "7824cc42ee0447a4a2fa56b9c93bbabb", # ['TrainingScene.txt']
    "95228b7595954054ae2d11cfbd7623f8", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "a5a31c4af00b4199ac4c8760bfaa469e", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "addd1085215d42b38184517a2dc95d8d", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "f4db5c989fa64b90b31d396943964c12", # ['TrainingScene.txt']
    "a33a91bbd197432c91a8156f88e46c08", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "206b42297be84c7087da6c93f46615bd", # ['Incomplete.txt', 'MountainRoad.txt', 'TrainingScene.txt']
    "cf45d8bef0b842e5bac1b7022cc3c354", # ['MountainRoad.txt', 'TrainingScene.txt']
    "e9ec1e6c22c640168130d2756426666c", # ['TrainingScene.txt']
    "989685d9ca454491b184984695786e20", # ['TrainingScene.txt']
    "8f9b8786312349639e8811f451054df0", # ['MountainRoad.txt']
    "39e38c2aa06c42d79e50f36b631f9652", # ---- Key error ---> files are empty for certain scenes
    "5e66bfb48366498aa83c364bf31c6059", # ---- Key error
    "3358f368d8694472a86c4770d4f804bc", # ---- Key error
    "95e8781012c94d37a222b5679724cb1d", # ---- Key error
    "8626976ce8b14752be340be9f1070724", # ---- Key error
    "ab79e465598844af94e9bfdfca116fc4", # ---- Key error
    "6f7a67231a16454dbd657d2f48443aac", # ---- Key error
    "2cae1d5a3c0c4872b2f397deb13ece71", # ---- Key error
    "8b27ff8591af4cd79067cf156d23ab95", # ---- Key error
    "8245ccbaba844032b08f7c33d9c4308a", # ---- Key error 
    "d093379ce1224f1bb24b9730da105927", # ---- Key error 
    "ff0cb5fdc1884204937eca64b52ee041", # ---- Key error
    "05086cebdd4c45a084ed3ce6b25cc9e6", # ---- Key error
    "4858c7aa9362421988e7c88d5adae755", # ---- Key error
    "66bdb97a653b48f98c02d5e764089c00", # ---- Key error
    "3100b15011da48d88fa3f7f053e4e19f", # ---- Key error
    "39e5235b13274feb88430f08f3cd5369", # ---- Key error
    "9b9220b8bf8c4e61adde8bc7571540ef", # ---- Key error
    "a57132e2aeba484e8433ae8af9b1abbc", # ---- Key error
    "3de6823af97441c18c8ca5f169b6e896", # ---- Key error
    "3d865974f89e44c583e23e4f3c8f3d14", # ---- Key error
    "011846a5c30c4325b45e439fc992f519", # ---- Key error
    "951592742a88497989fbb71a65781639", # ---- Key error 
    "867545b969cc4ff088b70eab632671dd", # ---- Key error
    "0d72afe467e74d4fb717a4ab9bb128ac", # ---- Key error
    "1d0296054e434021b3aa2b47b6c7d921", # ---- Number of supplied paths does not match 
    "0037b2329de444c18d751b4e79901b39", # ---- Calibration data seems to be missing
    "b89680ab48514db8bca8fcca1852f36d", # ---- Calibration data seems to be missing
    
    #####
    
    # These seem to later be causing nans, mostly likely due to interpolation of clusters 
    "81957954cd71450cacc5fd738dd9ebd2" # done, but has 3 segments where all values are 0 
    #"7e3a3b57dc2a436faf7a21f010206bb6" # done, but has 1 segment where all values are 0 
    "4a10fec1583843e5b1c82fb74fa7c775" # done, but has 7 segments where all values are 0
    "eadeee63de7f40a4b4375733651d5dd1" # done, but has 12 segments where all values are 0 
    #"8ba8fe140022448490326ad0d25ef5a3" # done, but has 2 segments where all values are 0 
    #"4702745fe0df4a778149aee6882a31de" # done, but has 1 segment where all values are 0
    "8e03eb1671774d3d9a35f97178902a45" # done, but has 7 segments where all values are 0
    
]

#exclude_participant_ids = []

In [ ]:
verbose = False

#participant_bootstrap_path = "/Volumes/LaCie/aMessdaten/LoopAR/Bootstrapped_Participants/"
participant_bootstrap_path = "./bootstrapped_participant_data/"
keep_participants_in_memory = False


eyes_paths  = ["/Volumes/LaCie/aMessdaten/LoopAR/Data_from_store/LoopAR Data/WestdriveLoopARData_left_complete/EyeTracking/","/Volumes/LaCie/aMessdaten/LoopAR/Data_from_store/LoopAR Data/WestdriveLoopARData_right_complete/EyeTracking/"]
input_paths = ["/Volumes/LaCie/aMessdaten/LoopAR/Data_from_store/LoopAR Data/WestdriveLoopARData_left_complete/Input/","/Volumes/LaCie/aMessdaten/LoopAR/Data_from_store/LoopAR Data/WestdriveLoopARData_right_complete/Input/"]
calib_paths = ["/Volumes/LaCie/aMessdaten/LoopAR/Data_from_store/LoopAR Data/WestdriveLoopARData_left_complete/ParticipantCalibrationData/","/Volumes/LaCie/aMessdaten/LoopAR/Data_from_store/LoopAR Data/WestdriveLoopARData_right_complete/ParticipantCalibrationData/"]
scene_paths = ["/Volumes/LaCie/aMessdaten/LoopAR/Data_from_store/LoopAR Data/WestdriveLoopARData_left_complete/SceneData/","/Volumes/LaCie/aMessdaten/LoopAR/Data_from_store/LoopAR Data/WestdriveLoopARData_right_complete/SceneData/"]

eyes_paths  = ["./data2/EyeTracking/"]
input_paths = ["./data2/Input/"]
calib_paths = ["./data2/ParticipantCalibrationData/"]
scene_paths = ["./data2/SceneData/"]


measurement_full = MeasurementData(eyes_paths,input_paths,calib_paths,scene_paths,participant_bootstrap_path,exclude_participant_ids,keep_participants_in_memory,verbose)


In [ ]:
print("Available Areas (in vanilla data; not multiplied by segments):")
print(measurement_full.get_total_available_areas())
print("Available Participants:",len(measurement_full.get_participant_list()))


In [ ]:
measurement_full.apply_reference_data(REFERENCE_DATA_ALL_EVENTS_FAILED)

In [ ]:
full_filtered_data = measurement_full.get_data(use_vanilla=False, filter_data=True,filter_by_corr_coeff_dict = CORRELATION_DICT, corr_coeff_threshold=0.95)

In [ ]:
full_vanilla_data = measurement_full.get_data(use_vanilla=True, filter_data=False)

In [ ]:
save_to_disk(full_filtered_data, final_data_path + "full_filtered_data.pickle")

In [ ]:
print("NaNs")

for participant in list(full_filtered_data.keys()):
    for area in list(full_filtered_data[participant].keys()):
        for segment in list(full_filtered_data[participant][area].keys()):
            
            if full_filtered_data[participant][area][segment].isnull().any().any():
                print(participant, area, segment)
                
                # remove 
                full_filtered_data[participant][area].pop(segment)
                
            

In [ ]:
westbrueck_cnt = 0
autobahn_cnt = 0
countryroad_cnt = 0
mountainroad_cnt = 0 


westbrueck_segments = [] # list of dfs 

for participant in full_filtered_data:
    for area in full_filtered_data[participant]:
        for segment in full_filtered_data[participant][area]:
            
            if area == "Autobahn":
                autobahn_cnt += 1
            if area == "CountryRoad":
                countryroad_cnt += 1
            if area == "MountainRoad":
                mountainroad_cnt += 1
            if area == "Westbrueck":
                westbrueck_cnt += 1
                westbrueck_segments.append(full_filtered_data[participant][area][segment])
                
save_to_disk(westbrueck_segments, final_data_path + "westbrueck_segments.pickle")

print("Available Segments in Areas (after filtering):")
print("Autobahn:",autobahn_cnt,"\tWestbrueck:",westbrueck_cnt,"\tCountryRoad:",countryroad_cnt,"\tMountainRoad:",mountainroad_cnt)
